In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

from miniMTL.datasets import *
from miniMTL.models import *
from miniMTL.util import *
from miniMTL.training import *

## Load data

In [2]:
p_pheno = '/home/harveyaa/Documents/fMRI/data/ukbb_9cohorts/pheno_01-12-21.csv'
p_conn = '/home/harveyaa/Documents/fMRI/data/ukbb_9cohorts/connectomes/'

cases = ['SZ',
        'BIP',
        #'ASD',
        #'DEL22q11_2',
        #'DEL16p11_2',
        # 'DUP16p11_2',
        # 'DUP22q11_2'
        ]
data = []
for case in cases:
    data.append(caseControlDataset(case,p_pheno,p_conn))

/home/harveyaa/miniconda3/envs/MTL/lib/python3.7/site-packages/ipykernel_launcher.py:14: DtypeWarning: Columns (7,8,12,13,14,19,20,24,27,31,42,43,44,49,51,59,60,62,64,65,68,101,121,163) have mixed types.Specify dtype option on import or set low_memory=False.
  


In [3]:
loss_fns = {}
trainloaders = {}
testloaders = {}
decoders = {}
for d, case in zip(data,cases):
    train_d, test_d = split_data(d)

    trainloaders[case] = DataLoader(train_d, batch_size=16, shuffle=True)
    testloaders[case] = DataLoader(test_d, batch_size=16, shuffle=True)
    loss_fns[case] = nn.CrossEntropyLoss()
    decoders[case] = head().double()

In [4]:
model = HPSModel(encoder().double(),
                decoders,
                loss_fns)

In [5]:
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
trainer = Trainer(optimizer,50)

trainer.fit(model,trainloaders,testloaders)

Epoch 49: 100%|██████████| 40/40 [00:02<00:00, 19.85it/s]


In [24]:
#TODO THIS IS BROKEN OBVY WANT A STABLE ANSWER

metrics = model.score(testloaders)
for key in metrics.keys():
    print(key)
    print('Accuracy: ', metrics[key]['accuracy'])
    print('Test loss: ', metrics[key]['test_loss'])
    print()

SZ
Accuracy:  46.875
Test loss:  0.04415733229872337

BIP
Accuracy:  53.125
Test loss:  0.043829105474226276

